In [1]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

%run 1.tokenizer.ipynb

tokenizer

In [2]:
%run 2.dataset.ipynb

dataset = Dataset(negative_label=False, with_answer=False)

len(dataset), dataset[0]

(1000000, {'text': '323+7939=8262+945=9207+2570=', 'label': 1})

In [3]:
%run 3.model.ipynb

model_actor = torch.load('model/ppo', weights_only=False).to(device)

In [4]:
#====question====
question = random.choices(dataset, k=8)
question = [i['text'] for i in question]

question = tokenizer(question,
                     device=device,
                     add_bos_token=True,
                     add_eos_token=False,
                     padding_side='left')['input_ids']

#====gen====
gen = generate(model_actor,
               input_ids=question,
               pad_token_id=tokenizer.pad_token_id,
               eos_token_id=tokenizer.eos_token_id)
gen = gen[:, question.shape[1]:]

last_index = lambda lst, ele: len(lst) - lst[::-1].index(ele)

for q, g in zip(question, gen):
    q = tokenizer.decode(q)
    g = tokenizer.decode(g)

    print({'question': q, 'gen': g})

{'question': 'B9818+4065=13883+7277=21160+9723=', 'gen': '30883E'}
{'question': 'B5354+5465=', 'gen': '10819E'}
{'question': 'B5823+9136=', 'gen': '14959E'}
{'question': 'B5197+8122=13319+6036=19355+5026=', 'gen': '24381E'}
{'question': 'B4619+4586=9205+9146=', 'gen': '18351E'}
{'question': 'B7590+472=8062+664=', 'gen': '8726E'}
{'question': 'B530+8908=9438+4916=', 'gen': '14354E'}
{'question': 'B1118+1419=2537+3398=5935+9187=', 'gen': '15122E'}


In [5]:
sent = ''


def inference(q):
    global sent

    sent += q + '='

    #限制最大轮次
    if sent.count('=') > 3:
        idx = sent.index('=') + 1
        sent = sent[idx:]

    #编码
    q = tokenizer([sent],
                  device=device,
                  add_bos_token=True,
                  add_eos_token=False)['input_ids']

    #生成回答
    gen = generate(model_actor,
                   input_ids=q,
                   pad_token_id=tokenizer.pad_token_id,
                   eos_token_id=tokenizer.eos_token_id)
    gen = gen[:, q.shape[1]:][0].tolist()

    #切除eos
    if tokenizer.eos_token_id in gen:
        idx = gen.index(tokenizer.eos_token_id)
        gen = gen[:idx]

    #如果生成了多余的对话,切除
    if tokenizer.eq_token_id in gen:
        idx = gen.index(tokenizer.eq_token_id)
        gen = gen[:idx]

    gen = tokenizer.decode(gen)

    #拼接上下文
    sent += gen

    return gen


inference('1000+1000')

'2000'

In [6]:
inference('+500')

'2500'

In [7]:
inference('+500')

'3000'

In [8]:
inference('+500')

'3500'